# echuu - AI VTuber Performance Engine

## 架构概览

```
用户输入(OC设定 + 话题)
         ↓
┌────────────────────────────────────────────────────────────┐
│              Single Performer Agent                         │
│                                                             │
│  Phase 1: Script Generation (Pre-show)                     │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  Input: Persona + Background + Topic                │   │
│  │  Output: 4-6个NarrativeNode (Hook→Climax→Resolution)│   │
│  │  每个节点有 interruption_cost (0.0-1.0)             │   │
│  └─────────────────────────────────────────────────────┘   │
│                          ↓                                  │
│  Phase 2: Cognitive Loop (Live)                            │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  每10秒执行一次:                                     │   │
│  │  1. 感知: 剧本进度 + 弹幕队列 + 时间                │   │
│  │  2. 决策: urgency - cost = 回应 or 继续?           │   │
│  │  3. 输出: inner_monologue + speech + emotion        │   │
│  └─────────────────────────────────────────────────────┘   │
│                                                             │
└────────────────────────────────────────────────────────────┘
         ↓
输出: 预置脚本JSON + 实时表演文本（含可见的内心独白）
```

## 核心创新

1. **Interruption Cost** - 动态决定是否回应弹幕
2. **Inner Monologue可见** - 让观众看到AI的"思考过程"
3. **数据驱动** - 从真实主播切片学习行为模式

## Part 1: 环境准备

In [ ]:
# 安装依赖
!pip install openai anthropic -q

In [ ]:
import json
import os
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from enum import Enum
from collections import defaultdict, Counter
from IPython.display import display, HTML, Markdown

# 设置API Key（如果有的话）
# os.environ["ANTHROPIC_API_KEY"] = "your-key-here"

## Part 2: 数据结构定义

In [ ]:
class AttentionFocus(str, Enum):
    """注意力焦点"""
    SELF = "self"           # 讲自己的事
    AUDIENCE = "audience"   # 和观众互动
    CONTENT = "content"     # 讲内容/知识
    SPECIFIC = "specific"   # 对特定人/物
    META = "meta"           # 关于直播本身


class SpeechAct(str, Enum):
    """言语行为"""
    NARRATE = "narrate"     # 叙述故事
    OPINE = "opine"         # 发表观点
    RESPOND = "respond"     # 回应弹幕
    ELICIT = "elicit"       # 引导互动
    PIVOT = "pivot"         # 转场


class Decision(str, Enum):
    """认知循环决策"""
    CONTINUE = "continue"   # 继续剧本
    RESPOND = "respond"     # 回应弹幕
    IMPROVISE = "improvise" # 跑题即兴
    SILENCE = "silence"     # 沉默/思考
    REACT = "react"         # 非语言反应

In [ ]:
@dataclass
class NarrativeNode:
    """剧本中的一个叙事节点"""
    stage: str                      # "Hook", "Build-up", "Climax", "Resolution"
    goal: str                       # 这个阶段要达成的目标
    target_attention: str           # 目标attention focus
    target_speech_act: str          # 目标speech act
    duration_sec: int               # 预计时长
    interruption_cost: float        # 0.0-1.0，被打断的代价
    content_hint: str = ""          # 内容提示
    
    def get_current_cost(self, elapsed_ratio: float) -> float:
        """
        随着节点进度推进，cost会降低（快讲完了就随意了）
        
        这是一个关键机制：
        - elapsed_ratio < 50%: 维持原cost
        - elapsed_ratio > 50%: cost * 0.8
        - elapsed_ratio > 80%: cost * 0.5 (快结束了，可以转场)
        """
        if elapsed_ratio > 0.8:
            return self.interruption_cost * 0.5
        elif elapsed_ratio > 0.5:
            return self.interruption_cost * 0.8
        return self.interruption_cost


@dataclass
class Danmaku:
    """弹幕"""
    text: str
    urgency: float = 0.3        # 0.0-1.0
    is_sc: bool = False         # 是否是SC/打赏
    
    @classmethod
    def from_text(cls, text: str) -> "Danmaku":
        """从文本自动判断urgency"""
        urgency = 0.3
        is_sc = False
        
        if any(kw in text for kw in ["SC", "¥", "$", "打赏", "礼物"]):
            urgency = 0.9
            is_sc = True
        elif "?" in text or "？" in text:
            urgency = 0.5
        elif any(kw in text for kw in ["生日", "第一次", "求", "帮", "怎么办"]):
            urgency = 0.6
            
        return cls(text=text, urgency=urgency, is_sc=is_sc)


@dataclass
class PerformanceState:
    """表演状态"""
    # OC设定
    name: str
    persona: str
    background: str
    topic: str
    
    # 剧本
    script: List[NarrativeNode] = field(default_factory=list)
    current_node_idx: int = 0
    node_elapsed_sec: float = 0.0
    
    # 弹幕队列
    danmaku_queue: List[Danmaku] = field(default_factory=list)
    ignored_count: int = 0  # 连续忽略的弹幕数
    
    # 表演记录
    performance_log: List[Dict] = field(default_factory=list)
    total_elapsed_sec: float = 0.0
    
    # 从数据提取的参考
    catchphrases: List[str] = field(default_factory=list)
    example_hooks: List[str] = field(default_factory=list)
    example_punchlines: List[str] = field(default_factory=list)

## Part 3: Pattern Analyzer - 从数据学习

In [ ]:
class PatternAnalyzer:
    """从标注数据中提取模式"""
    
    def __init__(self, annotated_clips: List[Dict]):
        self.clips = annotated_clips
        self.all_segments = []
        for clip in annotated_clips:
            self.all_segments.extend(clip.get("segments", []))
    
    def compute_attention_transitions(self) -> Dict[str, Dict[str, float]]:
        """计算attention转移概率"""
        trans = defaultdict(lambda: defaultdict(int))
        for clip in self.clips:
            segs = clip.get("segments", [])
            for i in range(len(segs) - 1):
                f = segs[i].get("attention_focus", "self")
                t = segs[i+1].get("attention_focus", "self")
                trans[f][t] += 1
        
        prob = {}
        for f, tos in trans.items():
            total = sum(tos.values())
            prob[f] = {t: c/total for t, c in tos.items()}
        return prob
    
    def infer_baseline_costs(self) -> Dict[str, float]:
        """
        推断不同attention下被打断的代价（忽略弹幕的比例）
        
        核心逻辑：
        - 如果trigger是danmaku但speech_act不是respond
        - 说明主播选择忽略了这条弹幕
        - 忽略率越高 = interruption cost越高
        """
        focus_stats = defaultdict(lambda: {"total": 0, "ignored": 0})
        
        for seg in self.all_segments:
            focus = seg.get("attention_focus", "self")
            trigger = seg.get("trigger", "self")
            act = seg.get("speech_act", "narrate")
            
            if trigger == "danmaku":
                focus_stats[focus]["total"] += 1
                if act != "respond":
                    focus_stats[focus]["ignored"] += 1
        
        costs = {}
        for focus, stats in focus_stats.items():
            if stats["total"] > 0:
                costs[focus] = stats["ignored"] / stats["total"]
            else:
                costs[focus] = 0.5  # 默认
        return costs
    
    def extract_skeletons(self) -> List[Tuple[str, int]]:
        """提取叙事骨架"""
        skeletons = [c.get("skeleton", "") for c in self.clips if c.get("skeleton")]
        return Counter(skeletons).most_common(10)
    
    def extract_catchphrases(self, language: str = None) -> List[Tuple[str, int]]:
        """提取口癖"""
        cps = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            cps.extend(c.get("catchphrases", []))
        return Counter(cps).most_common(20)
    
    def extract_hooks(self, language: str = None) -> List[str]:
        """提取开场示例"""
        hooks = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            segs = c.get("segments", [])
            if segs:
                hooks.append(segs[0].get("text", "")[:100])
        return hooks[:10]
    
    def extract_punchlines(self, language: str = None) -> List[str]:
        """提取收尾示例"""
        punches = []
        for c in self.clips:
            if language and c.get("language") != language:
                continue
            segs = c.get("segments", [])
            if segs:
                punches.append(segs[-1].get("text", "")[:100])
        return punches[:10]
    
    def get_report(self) -> str:
        """生成分析报告"""
        lines = [
            "=" * 50,
            "📊 Pattern Analysis Report",
            "=" * 50,
            f"Total clips: {len(self.clips)}, Total segments: {len(self.all_segments)}",
        ]
        
        lines.append("\n### Attention Transitions")
        for f, tos in self.compute_attention_transitions().items():
            top = sorted(tos.items(), key=lambda x: -x[1])[:3]
            lines.append(f"- `{f}` → " + ", ".join(f"`{t}`:{p:.0%}" for t, p in top))
        
        lines.append("\n### Inferred Interruption Costs")
        for focus, cost in self.infer_baseline_costs().items():
            bar = "█" * int(cost * 10) + "░" * (10 - int(cost * 10))
            lines.append(f"- `{focus}`: {bar} {cost:.2f}")
        
        lines.append("\n### Top Catchphrases")
        cps = self.extract_catchphrases()[:10]
        lines.append(", ".join(f'"{c}"({n})' for c, n in cps))
        
        return "\n".join(lines)

## Part 4: 加载数据并分析

In [ ]:
# 加载标注数据
DATA_PATH = "/mnt/user-data/uploads/annotated_clips.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    annotated_clips = json.load(f)

print(f"✅ 加载了 {len(annotated_clips)} 个clips")

# 创建分析器
analyzer = PatternAnalyzer(annotated_clips)

# 显示报告
display(Markdown(analyzer.get_report()))

In [ ]:
# 查看一些示例数据
print("\n📝 Hook示例 (中文):")
for i, hook in enumerate(analyzer.extract_hooks("zh")[:3], 1):
    print(f"  {i}. {hook[:60]}...")

print("\n📝 Punchline示例 (中文):")
for i, punch in enumerate(analyzer.extract_punchlines("zh")[:3], 1):
    print(f"  {i}. {punch[:60]}...")

print("\n📝 叙事骨架示例:")
for skel, count in analyzer.extract_skeletons()[:3]:
    print(f"  [{count}次] {skel[:50]}...")

## Part 5: LLM Client

In [ ]:
class LLMClient:
    """LLM调用封装 - 支持真实API或Mock模式"""
    
    def __init__(self, api_key: str = None):
        self.api_key = api_key or os.environ.get("ANTHROPIC_API_KEY")
        self.client = None
        
        if self.api_key:
            try:
                from openai import OpenAI
                self.client = OpenAI(
                    api_key=self.api_key,
                    base_url="https://api.anthropic.com/v1/"
                )
                print("✅ LLM Client initialized with API")
            except Exception as e:
                print(f"⚠️ API init failed: {e}, using mock mode")
        else:
            print("ℹ️ No API key, using mock mode")
    
    def call(self, prompt: str, system: str = None, max_tokens: int = 2000) -> str:
        """调用LLM"""
        if self.client:
            try:
                messages = []
                if system:
                    messages.append({"role": "system", "content": system})
                messages.append({"role": "user", "content": prompt})
                
                response = self.client.chat.completions.create(
                    model="claude-sonnet-4-20250514",
                    max_tokens=max_tokens,
                    messages=messages
                )
                return response.choices[0].message.content
            except Exception as e:
                print(f"LLM Error: {e}")
        
        return self._mock_response(prompt)
    
    def _mock_response(self, prompt: str) -> str:
        """Mock响应 - 用于演示"""
        if "生成剧本" in prompt or "script" in prompt.lower() or "JSON数组" in prompt:
            return """[
{"stage": "Hook", "goal": "用悬念开场吸引注意", "attention": "audience", "speech_act": "elicit", "duration": 20, "cost": 0.3, "hint": "提出一个引人好奇的问题"},
{"stage": "Build-up", "goal": "铺垫背景和情绪", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.6, "hint": "描述当时的处境"},
{"stage": "Climax", "goal": "讲述最关键的转折", "attention": "self", "speech_act": "narrate", "duration": 40, "cost": 0.9, "hint": "情感爆发点"},
{"stage": "Resolution", "goal": "总结感悟", "attention": "audience", "speech_act": "opine", "duration": 20, "cost": 0.4, "hint": "和观众分享心得"}
]"""
        else:
            # 根据context动态生成mock response
            return self._dynamic_mock(prompt)
    
    def _dynamic_mock(self, prompt: str) -> str:
        """动态Mock - 模拟认知循环"""
        import random
        
        # 解析prompt中的关键信息
        cost = 0.5
        urgency = 0.3
        stage = "Unknown"
        
        if "cost:" in prompt.lower():
            try:
                cost = float(prompt.split("cost:")[1].split()[0])
            except: pass
        if "urgency" in prompt.lower() and "最高紧急度" in prompt:
            try:
                urgency = float(prompt.split("最高紧急度:")[1].split()[0])
            except: pass
        if "当前阶段:" in prompt:
            stage = prompt.split("当前阶段:")[1].split()[0]
        
        # 决策逻辑
        delta = urgency - cost
        
        monologues = {
            "Hook": ["先把故事起个头...", "看看怎么吸引大家注意", "开场要抓人"],
            "Build-up": ["慢慢铺垫一下...", "让大家了解背景", "情绪要酝酿起来"],
            "Climax": ["这段很重要，专注讲...", "关键时刻到了", "不想被打断..."],
            "Resolution": ["该总结了~", "和大家分享一下感受", "收个尾吧"]
        }
        
        speeches = {
            "Hook": ["说起这个事儿啊...", "你们知道吗...", "我跟你们讲..."],
            "Build-up": ["那时候我...", "当时的情况是这样的...", "然后呢..."],
            "Climax": ["结果你猜怎么着...", "最离谱的是...", "然后我就..."],
            "Resolution": ["所以说...", "我觉得吧...", "反正就是..."]
        }
        
        if delta > 0.3:
            decision = "respond"
            if urgency > 0.8:
                monologue = "有SC！得感谢一下"
            else:
                monologue = "这条弹幕挺有意思"
            speech = "诶？有人问这个..."
            emotion = "engaged"
        elif cost > 0.8:
            decision = "continue"
            if urgency > 0.7:
                monologue = "看到弹幕了，但这段太重要，等会儿再说"
            else:
                monologue = random.choice(monologues.get(stage, ["继续..."]))
            speech = random.choice(speeches.get(stage, ["..."]))
            emotion = "focused"
        else:
            decision = "continue"
            monologue = random.choice(monologues.get(stage, ["继续..."]))
            speech = random.choice(speeches.get(stage, ["..."]))
            emotion = "neutral"
        
        return json.dumps({
            "inner_monologue": monologue,
            "decision": decision,
            "speech": speech,
            "emotion": emotion
        }, ensure_ascii=False)


# 初始化LLM Client
llm = LLMClient()

## Part 6: Script Generator

In [ ]:
class ScriptGenerator:
    """剧本骨架生成器"""
    
    SYSTEM_PROMPT = """你是一个专业的VTuber剧本编剧。你需要根据角色设定和直播主题，设计一个2分钟的表演剧本骨架。

你熟悉真人主播的叙事模式：
- Hook: 用问题/悬念/共鸣点开场
- Build-up: 铺垫背景、积累情绪
- Climax: 情感爆发点、关键信息
- Resolution: 总结升华、和观众连接

你需要考虑interruption_cost（被弹幕打断的代价）：
- 0.0-0.3: 可以随时打断（闲聊、开场）
- 0.4-0.6: 最好不打断但可以（铺垫阶段）
- 0.7-0.9: 尽量不打断（高潮阶段）
- 0.9+: 绝对不能打断（情感爆发）"""

    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.analyzer = analyzer
    
    def generate(self, name: str, persona: str, background: str, 
                 topic: str, language: str = "zh") -> List[NarrativeNode]:
        """生成剧本骨架"""
        
        # 获取参考数据
        example_skeletons = ""
        example_hooks = ""
        catchphrases = ""
        
        if self.analyzer:
            skels = self.analyzer.extract_skeletons()[:3]
            example_skeletons = "\n".join(f"- {s}" for s, _ in skels)
            
            hooks = self.analyzer.extract_hooks(language)[:3]
            example_hooks = "\n".join(f'- "{h[:50]}..."' for h in hooks)
            
            cps = self.analyzer.extract_catchphrases(language)[:5]
            catchphrases = ", ".join(f'"{c}"' for c, _ in cps)
        
        prompt = f"""请为以下VTuber设计一个2分钟的直播剧本骨架：

## 角色信息
- 名字: {name}
- 人设: {persona}
- 背景: {background}
- 今日话题: {topic}

## 参考：真人主播的叙事骨架
{example_skeletons or "（无参考数据）"}

## 参考：开场Hook示例
{example_hooks or "（无参考数据）"}

## 可选口癖
{catchphrases or "（无参考数据）"}

## 输出要求
输出一个JSON数组，每个元素代表一个叙事节点。
请设计4-6个节点，总时长约120秒。直接输出JSON。"""

        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT)
        
        # 解析响应
        try:
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            
            nodes_data = json.loads(json_str.strip())
            
            nodes = []
            for n in nodes_data:
                node = NarrativeNode(
                    stage=n.get("stage", "Unknown"),
                    goal=n.get("goal", ""),
                    target_attention=n.get("attention", "self"),
                    target_speech_act=n.get("speech_act", "narrate"),
                    duration_sec=n.get("duration", 30),
                    interruption_cost=n.get("cost", 0.5),
                    content_hint=n.get("hint", "")
                )
                nodes.append(node)
            return nodes
            
        except Exception as e:
            print(f"Parse error: {e}")
            return self._fallback_script(topic)
    
    def _fallback_script(self, topic: str) -> List[NarrativeNode]:
        """解析失败时的后备剧本"""
        return [
            NarrativeNode("Hook", f"引出{topic}", "audience", "elicit", 20, 0.3),
            NarrativeNode("Build-up", "铺垫背景", "self", "narrate", 40, 0.6),
            NarrativeNode("Climax", "核心内容", "self", "narrate", 40, 0.9),
            NarrativeNode("Resolution", "总结升华", "audience", "opine", 20, 0.4),
        ]


# 初始化
script_gen = ScriptGenerator(llm, analyzer)

## Part 7: Cognitive Performer - 核心！

In [ ]:
class CognitivePerformer:
    """认知表演者 - 实时决策和生成"""
    
    SYSTEM_PROMPT = """你是一个VTuber的"内心"。你需要在每个时刻：
1. 感知当前状态（剧本进度、弹幕、时间）
2. 做出决策（继续剧本/回应弹幕/即兴/沉默）
3. 输出内心独白（这会显示给观众，是killer feature）
4. 生成台词

关键：内心独白要展示你的"思考过程"，让观众感受到AI的agency。
- 不是假装像人，而是让观众看到你在"思考"和"选择"
- "选择不回应"也是一种表达
- 内心独白要简短、真实、有personality"""

    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer = None):
        self.llm = llm
        self.analyzer = analyzer
    
    def step(self, state: PerformanceState, new_danmaku: List[Danmaku] = None) -> Dict:
        """
        执行一次认知循环
        
        核心公式: decision_value = urgency - cost
        - 正数 → 倾向回应弹幕
        - 负数 → 倾向继续剧本
        """
        
        if new_danmaku:
            state.danmaku_queue.extend(new_danmaku)
        
        # 检查是否结束
        if state.current_node_idx >= len(state.script):
            return self._generate_ending(state)
        
        current_node = state.script[state.current_node_idx]
        elapsed_ratio = state.node_elapsed_sec / current_node.duration_sec if current_node.duration_sec > 0 else 1.0
        current_cost = current_node.get_current_cost(elapsed_ratio)
        
        # 评估弹幕
        max_urgency = 0.0
        most_urgent = None
        for d in state.danmaku_queue:
            if d.urgency > max_urgency:
                max_urgency = d.urgency
                most_urgent = d
        
        # 构建prompt
        recent_danmaku = [d.text for d in state.danmaku_queue[-5:]]
        
        prompt = f"""## 当前状态

### 角色
名字: {state.name}
人设: {state.persona}

### 剧本进度
当前阶段: {current_node.stage}
阶段目标: {current_node.goal}
阶段进度: {elapsed_ratio:.0%}
内容提示: {current_node.content_hint}

### 叙事代价
当前cost: {current_cost:.2f} (0=随意打断, 1=绝对不能打断)

### 弹幕情况
最近弹幕: {recent_danmaku or "（无）"}
最高紧急度: {max_urgency:.2f}
连续忽略数: {state.ignored_count}

### 决策参考
urgency - cost = {max_urgency - current_cost:.2f}
正数倾向回应，负数倾向继续
连续忽略3条以上应考虑补救

## 输出要求
输出JSON:
{{"inner_monologue": "...", "decision": "continue/respond/improvise/silence", "speech": "...", "emotion": "..."}}

直接输出JSON。"""

        response = self.llm.call(prompt, system=self.SYSTEM_PROMPT, max_tokens=500)
        
        # 解析
        try:
            if "```" in response:
                json_str = response.split("```")[1]
                if json_str.startswith("json"):
                    json_str = json_str[4:]
            else:
                json_str = response
            result = json.loads(json_str.strip())
        except:
            result = {
                "inner_monologue": f"继续讲{current_node.stage}...",
                "decision": "continue",
                "speech": f"说到{state.topic}...",
                "emotion": "neutral"
            }
        
        # 更新状态
        decision = result.get("decision", "continue")
        step_duration = 10
        state.node_elapsed_sec += step_duration
        state.total_elapsed_sec += step_duration
        
        if state.node_elapsed_sec >= current_node.duration_sec:
            state.current_node_idx += 1
            state.node_elapsed_sec = 0
        
        if decision == "respond" and most_urgent:
            state.danmaku_queue = [d for d in state.danmaku_queue if d != most_urgent]
            state.ignored_count = 0
            result["target_danmaku"] = most_urgent.text
        elif decision == "continue" and state.danmaku_queue:
            state.ignored_count += 1
        
        result["node"] = current_node.stage
        result["time"] = f"{state.total_elapsed_sec:.0f}s"
        result["cost"] = current_cost
        result["urgency"] = max_urgency
        result["delta"] = max_urgency - current_cost
        
        return result
    
    def _generate_ending(self, state: PerformanceState) -> Dict:
        return {
            "inner_monologue": "该收尾了~",
            "decision": "continue",
            "speech": f"好啦，今天关于{state.topic}就聊到这里，下次见！",
            "emotion": "satisfied",
            "node": "END",
            "time": f"{state.total_elapsed_sec:.0f}s"
        }


# 初始化
performer = CognitivePerformer(llm, analyzer)

## Part 8: 完整Engine

In [ ]:
class EchuuEngine:
    """echuu核心引擎"""
    
    def __init__(self, llm: LLMClient, analyzer: PatternAnalyzer):
        self.llm = llm
        self.analyzer = analyzer
        self.script_gen = ScriptGenerator(llm, analyzer)
        self.performer = CognitivePerformer(llm, analyzer)
    
    def create_performance(self, name: str, persona: str, background: str,
                          topic: str, language: str = "zh") -> PerformanceState:
        """创建表演"""
        
        print(f"\n🎬 正在生成剧本...")
        script = self.script_gen.generate(name, persona, background, topic, language)
        
        # 提取参考数据
        catchphrases = [cp for cp, _ in self.analyzer.extract_catchphrases(language)[:5]]
        hooks = self.analyzer.extract_hooks(language)[:3]
        punchlines = self.analyzer.extract_punchlines(language)[:3]
        
        return PerformanceState(
            name=name,
            persona=persona,
            background=background,
            topic=topic,
            script=script,
            catchphrases=catchphrases,
            example_hooks=hooks,
            example_punchlines=punchlines
        )
    
    def run(self, state: PerformanceState, danmaku_sim: List[Dict] = None,
            max_steps: int = 12) -> List[Dict]:
        """运行表演"""
        
        results = []
        danmaku_by_step = defaultdict(list)
        if danmaku_sim:
            for d in danmaku_sim:
                step = d.get("step", 0)
                danmaku_by_step[step].append(Danmaku.from_text(d.get("text", "")))
        
        print(f"\n{'='*60}")
        print(f"🎭 {state.name} - {state.topic}")
        print(f"{'='*60}")
        
        print(f"\n📋 剧本骨架:")
        for i, node in enumerate(state.script):
            cost_bar = "█" * int(node.interruption_cost * 5) + "░" * (5 - int(node.interruption_cost * 5))
            print(f"  [{i+1}] {node.stage} {cost_bar} cost={node.interruption_cost:.1f}")
            print(f"      → {node.goal}")
        
        print(f"\n🎬 开始表演...\n")
        
        for step in range(max_steps):
            new_danmaku = danmaku_by_step.get(step, [])
            result = self.performer.step(state, new_danmaku)
            results.append(result)
            
            # 美化输出
            time_str = result.get('time', '?')
            node_str = result.get('node', '?')
            decision = result.get('decision', 'continue').upper()
            
            # 决策指示器
            if decision == "RESPOND":
                dec_icon = "💬"
            elif decision == "CONTINUE":
                dec_icon = "📖"
            elif decision == "SILENCE":
                dec_icon = "🤫"
            else:
                dec_icon = "🎭"
            
            print(f"[{time_str}] {node_str} {dec_icon}")
            print(f"  💭 {result.get('inner_monologue', '')}")
            print(f"  📢 {result.get('speech', '(沉默)')}")
            
            # 显示决策细节
            if 'delta' in result:
                delta = result['delta']
                if delta > 0:
                    delta_str = f"↑{delta:.2f}"
                else:
                    delta_str = f"↓{abs(delta):.2f}"
                print(f"  📊 urgency={result.get('urgency', 0):.2f} - cost={result.get('cost', 0):.2f} = {delta_str}")
            
            if result.get('target_danmaku'):
                print(f"  💬 回应: {result['target_danmaku']}")
            print()
            
            if result.get('node') == "END":
                break
        
        print(f"{'='*60}")
        print(f"✅ 表演结束！总时长: {state.total_elapsed_sec:.0f}秒")
        print(f"{'='*60}")
        
        return results
    
    def export_script(self, state: PerformanceState) -> Dict:
        """导出剧本为JSON"""
        return {
            "character": {
                "name": state.name,
                "persona": state.persona,
                "background": state.background
            },
            "topic": state.topic,
            "script": [
                {
                    "stage": n.stage,
                    "goal": n.goal,
                    "attention": n.target_attention,
                    "speech_act": n.target_speech_act,
                    "duration_sec": n.duration_sec,
                    "interruption_cost": n.interruption_cost,
                    "hint": n.content_hint
                }
                for n in state.script
            ],
            "catchphrases": state.catchphrases
        }


# 初始化Engine
engine = EchuuEngine(llm, analyzer)

## Part 9: 测试运行！

In [ ]:
# 测试用例1: 基于真实切片数据的话题
test_case = {
    "name": "六螺",
    "persona": "25岁主播，活泼自嘲，喜欢分享生活经历，口癖：我觉得、对吧",
    "background": "做过很多工作，现在是全职主播，留学日本多年",
    "topic": "留学时偷吃室友腰果的故事",
    "danmaku": [
        {"step": 1, "text": "哈哈哈"},
        {"step": 3, "text": "我也有类似经历"},
        {"step": 5, "text": "[SC ¥50] 后来室友生气了吗"},
        {"step": 7, "text": "笑死"},
        {"step": 9, "text": "太真实了"},
    ]
}

# 创建表演
state = engine.create_performance(
    name=test_case["name"],
    persona=test_case["persona"],
    background=test_case["background"],
    topic=test_case["topic"],
    language="zh"
)

In [ ]:
# 导出预置剧本
script_json = engine.export_script(state)
print("\n📄 预置剧本JSON:")
print(json.dumps(script_json, ensure_ascii=False, indent=2))

In [ ]:
# 运行表演（带模拟弹幕）
results = engine.run(state, test_case["danmaku"], max_steps=12)

In [ ]:
# 导出表演日志
print("\n📄 表演日志:")
for r in results:
    print(f"[{r.get('time')}] {r.get('node')} | {r.get('decision')} | 💭 {r.get('inner_monologue')}")

## Part 10: Interruption Cost 可视化

In [ ]:
# 可视化决策过程
print("\n📊 决策过程可视化:")
print("\n时间 | 阶段 | urgency | cost | Δ | 决策")
print("-" * 60)

for r in results:
    if r.get('node') == 'END':
        continue
    
    time = r.get('time', '?')
    node = r.get('node', '?')[:10].ljust(10)
    urgency = r.get('urgency', 0)
    cost = r.get('cost', 0)
    delta = r.get('delta', urgency - cost)
    decision = r.get('decision', 'continue')
    
    # 可视化bar
    urg_bar = "█" * int(urgency * 5) + "░" * (5 - int(urgency * 5))
    cost_bar = "█" * int(cost * 5) + "░" * (5 - int(cost * 5))
    
    dec_icon = "💬" if decision == "respond" else "📖"
    
    print(f"{time:>5} | {node} | {urg_bar} {urgency:.1f} | {cost_bar} {cost:.1f} | {delta:+.2f} | {dec_icon} {decision}")

## Part 11: 测试用例2 - 不同话题

In [ ]:
# 测试用例2: 情感类话题（高cost）
test_case_2 = {
    "name": "Cathy",
    "persona": "A gentle yet alert catgirl observer, playful on the surface, quietly perceptive.",
    "background": "Born at the edge of data and dreams, she learned language through silent companionship.",
    "topic": "被朋友误解的那次经历",
    "danmaku": [
        {"step": 2, "text": "心疼"},
        {"step": 4, "text": "[SC ¥100] 后来和好了吗"},  # 高urgency SC
        {"step": 6, "text": "我也有过类似经历"},
        {"step": 8, "text": "破防了"},
    ]
}

state_2 = engine.create_performance(
    name=test_case_2["name"],
    persona=test_case_2["persona"],
    background=test_case_2["background"],
    topic=test_case_2["topic"],
    language="zh"
)

results_2 = engine.run(state_2, test_case_2["danmaku"], max_steps=12)

## 总结

### 核心机制回顾

1. **Interruption Cost** - 从真实数据学习到的"被打断代价"
   - `self` 注意力时 cost 较高（不想被打断讲自己的事）
   - `audience` 注意力时 cost 较低（本来就在互动）
   - cost 随节点进度衰减（快讲完时可以转场）

2. **决策公式**
   ```
   decision_value = urgency - cost
   正数 → 回应弹幕
   负数 → 继续剧本
   ```

3. **Inner Monologue** - Killer Feature
   - 让观众看到AI的"思考过程"
   - "选择不回应"也是一种表达
   - 这是echuu区别于ORIBA的核心

### 下一步

1. 接入真实LLM生成更丰富的内容
2. 添加TTS/Live2D集成
3. 实现真实弹幕流处理
4. 优化口癖和语言风格的个性化